In [1]:
import pygame
import cv2
import numpy
print(cv2.__version__)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
4.1.0


In [2]:
##image taking function
import cv2
import sys

cam = cv2.VideoCapture(0)
def takeImage():
    check, frame = cam.read()

    #im_ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
    #cv2.imwrite('hsu.png', im_ycrcb)
    #skin_ycrcb_mint = numpy.array((0, 133, 77))
    #skin_ycrcb_maxt = numpy.array((255, 173, 127))
    #skin_ycrcb = cv2.inRange(im_ycrcb, skin_ycrcb_mint, skin_ycrcb_maxt)
    #cv2.imwrite(sys.argv[2], skin_ycrcb) # Second image
    #print(skin_ycrcb_maxt)
    #contours, _ = cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL,
        #cv2.CHAIN_APPROX_SIMPLE)
    #for i, c in enumerate(contours):
        #area = cv2.contourArea(c)
        #if area > 1000:
            #cv2.drawContours(frame, contours, i, (255, 0, 0), 3)
    #cv2.imwrite('test.png', frame)
    # Final image
    cv2.imwrite("test.png", frame)
takeImage()
#cam.release()

In [3]:
##NN

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

# Function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    
    if checkpoint['arch'] == 'vgg16':
        
        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
    else:
        print("Architecture not recognized.")
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 6000)),
                                            ('relu', nn.ReLU()),
                                            ('drop', nn.Dropout(p=0.5)),
                                            ('fc2', nn.Linear(6000, 12)),
                                            ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

model = load_checkpoint('NNs/ABCDEFGHIdns4Gextra1.pth')
model.cuda()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [4]:
import json

with open('jsons/middle21.json', 'r') as f:
    number_to_ASL = json.load(f)


In [5]:
from PIL import Image

def process_image(image_path):
    # makes numpy image
    

    
    pil_image = Image.open(image_path)
    
    # Resize
    if pil_image.size[0] > pil_image.size[1]:
        pil_image.thumbnail((5000, 256))
    else:
        pil_image.thumbnail((256, 5000))
        
    # Crop 
    left_margin = (pil_image.width-224)/2
    bottom_margin = (pil_image.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    
    pil_image = pil_image.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    pil_image.save('cropped.png')
    
    # Normalize
    np_image = np.array(pil_image)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std
    

    np_image = np_image.transpose((2, 0, 1))
    
    return np_image

def predict(image_path, model, topk=3):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    image = process_image(image_path)
    
    # Convert image to PyTorch tensor first
    image = torch.from_numpy(image).type(torch.cuda.FloatTensor)
    #print(image.shape)
    #print(type(image))
    
    # Returns a new tensor with a dimension of size one inserted at the specified position.
    image = image.unsqueeze(0)
    
    output = model.forward(image)
    
    probabilities = torch.exp(output)
    
    # Probabilities and the indices of those probabilities corresponding to the classes
    top_probabilities, top_indices = probabilities.topk(topk)
    
    # Convert to lists
    top_probabilities = top_probabilities.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    top_indices = top_indices.detach().type(torch.FloatTensor).numpy().tolist()[0] 
    
    # Convert topk_indices to the actual class labels using class_to_idx
    # Invert the dictionary so you get a mapping from index to class.
    
    idx_to_class = {value: key for key, value in model.class_to_idx.items()}
    #print(idx_to_class)
    
    top_classes = [idx_to_class[index] for index in top_indices]
    
    return top_probabilities, top_classes


In [6]:
#Function to detect the actual sentence
longer = False
def detectSentence(hist, sentence):
    global longer
    last_el = hist[len(hist) - 1]
    same = True
    for element in hist:
        if element != last_el:
            same = False
            longer = False
    
    if same == True and longer == False:
        if last_el == "nothing":
            pass
        elif last_el == "del":
            sentence = sentence[:-1]
        elif last_el == "space":
            sentence = sentence + ' '
        else:
            sentence = sentence + last_el
        longer = True
    
    
    return sentence


##the pygame window 
pygame.init() 
pygame.font.init()

myfont = pygame.font.SysFont('Comic Sans MS', 30)
white = (255, 255, 255) 

X = 1000
Y = 800

display_surface = pygame.display.set_mode((X, Y ))
run = True
pygame.display.set_caption('ASL') 

hist = []
sentence = ''
while run == True : 

    display_surface.fill(white) 
    
    probs, classes = predict('test.png', model) 
    #append last prediction to hist
    hist.append(str(number_to_ASL[classes[0]]))
    text = str(number_to_ASL[classes[0]]) + ': ' + str(probs[0]) + '    ' + str(number_to_ASL[classes[1]]) + ': ' + str(probs[1])
    textsurface = myfont.render(text, False, (0, 0, 0))
    display_surface.blit(textsurface,(50, 500))
    #detect sentence
    if len(hist)< 20: 
        sentence =detectSentence(hist, sentence)
    else:
        sentence = detectSentence(hist[len(hist)-20:], sentence)
    sentsurface = myfont.render(sentence, False, (0, 0, 0))
    display_surface.blit(sentsurface,(50, 580))
    
    image = pygame.image.load(r'test.png')
    display_surface.blit(image, (0, 0)) 
    
    cropped_image = pygame.image.load(r'cropped.png')
    display_surface.blit(cropped_image, (700,0))
    
    pygame.display.update() 
    takeImage()
    
    
    
    for event in pygame.event.get() : 
        if event.type == pygame.QUIT : 
  
            pygame.quit() 
  
            quit() 
            cam.release()
            run = False
     
        
        
